In [ ]:
import numpy as np
from TMDP import TMDP
from algorithms import *
from model_functions import *

import matplotlib.pyplot as plt
import gymnasium as gym
from FrozenLake import *

#np.set_printoptions(precision=4)
import math

import torch
import mlflow
from CurriculumPPO import CurriculumPPO
import optuna
from policy_utils import *
from constants import *
from experiment_result_utils import *

nrows = 30
nS = nrows**2
nA = 4
map_seed = constants.SEEDS[6]


gamma = .99999999

is_slippery = False
debug = False
num_bins = 0
if num_bins == 0:
    reward_shape = dense_reward = False
    discount_tau = True
else:
    reward_shape = dense_reward = True
    discount_tau = False

checkpoint_step=500
test_episodes = 1000
episodes = 4900000
num_runs = 40


shape_range=(-1,0)

env = FrozenLakeEnv(is_slippery=False, seed=None, 
                    desc=generate_random_map(nrows, seed=map_seed), 
                    reward_shape=reward_shape,
                    num_bins=num_bins,
                    goal_reward=1,
                    shape_range=shape_range,
                    dense_reward=dense_reward,
                    )

xi = np.ones(env.nS) * 1/env.nS
tau = 1.
tmdp = TMDP(env, xi, tau=tau, gamma=gamma, seed=None, discount_tau=discount_tau)

In [ ]:

def get_or_create_experiment(experiment_name):
    if experiment := mlflow.get_experiment_by_name(experiment_name):
        return experiment.experiment_id
    else:
        return mlflow.create_experiment(experiment_name)


In [ ]:
mlflow.set_tracking_uri(MLFLOW_URI)
experiment_name = "OPTUNA"
experiment_id = get_or_create_experiment(experiment_name)
mlflow.set_experiment(experiment_name)

In [ ]:

final_temp=1-3
param_decay=True
lam = 1
batch_size = 32
epochs = 3

eps_ppo = 0.2
eps_model = compute_eps_model(gamma, tau, episodes/10)

In [ ]:

def objective(trial):
    with mlflow.start_run(nested=True):
        global episodes
        # Parameter space optimization
        model_lr = trial.suggest_float("model_lr", 1e-4, 0.5, log=True)
        pol_lr = trial.suggest_float("pol_lr", 1e-4, 0.5, log=True)
        batch_size = trial.suggest_int("batch_size", 1, 32, log=False)
        lam = trial.suggest_float("lam", 0., 1., log=False)
        temp = trial.suggest_float("temp", 0.1, 20, log=False)

        # Curriculum class initialization
        curr_PPO = CurriculumPPO(tmdp)
        
        tmdp.update_tau(tau) # Reset the teleport rate 
        
        curr_PPO.train(model_lr, pol_lr, batch_size=batch_size, 
                lam=lam, temp=temp, final_temp=final_temp,
                episodes=episodes, eps_ppo=eps_ppo,
                eps_model=eps_model, debug=debug,
                param_decay=param_decay, epochs=epochs)
        
        
        avg_reward = np.mean(curr_PPO.reward_records[-10:])/batch_size
        test_pol = test_policies_len(tmdp, [curr_PPO.theta], test_episodes, 1e-20)
        cum_return = test_pol[0][0]
        ep_len = test_pol[1][0]

        mlflow.log_params({
            "model_lr": model_lr,
            "pol_lr": pol_lr,
            "batch_size": batch_size,
            "lam": lam,
            "temp": temp,
            "final_temp": final_temp,
            "episodes": episodes,
            "tau": tau
        })

        mlflow.log_metric("Avg Reward", avg_reward)
        mlflow.log_metric("Cumulative Return", cum_return)
        mlflow.log_metric("Episode Length", ep_len)

        mlflow.set_tags(tags={
        "Project": "Curriculum Learning",
        "Algorithm": "CurriculumPPO",
        "Environment": tmdp.env.__class__.__name__,
        "nS": nS,
        "nA": nA,
        "gamma": gamma,
        "mu": tmdp.env.mu,
        "xi": tmdp.xi,
        "seed": seed,
        "tau": tau,
        "episodes": curr_PPO.episodes,
        })
         # Save the model to MLflow

    return cum_return


In [ ]:
run_name = "CPPO_RiverSwim_{}".format(nS)
with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True):
    if num_bins == 0:
        study = optuna.create_study(direction="maximize")
    else:
        study = optuna.create_study(direction="minimize")
        
    study.optimize(objective, n_trials=20)
    
    mlflow.log_params(study.best_params)
    
    # Train and save the best model
    curr_PPO = CurriculumPPO(tmdp)
    tmdp.update_tau(tau) # Reset the teleport rate 

    # Get the best parameters
    model_lr = study.best_params["model_lr"]
    pol_lr = study.best_params["pol_lr"]
    batch_size = study.best_params["batch_size"]
    lam = study.best_params["lam"]
    temp = study.best_params["temp"]
    
    # Train the best model
    curr_PPO.train(model_lr, pol_lr, batch_size=batch_size, 
                lam=lam, temp=temp, final_temp=final_temp,
                episodes=episodes, eps_ppo=eps_ppo,
                eps_model=eps_model, debug=debug,
                param_decay=param_decay, epochs=epochs)
    
    mlflow.set_tags(tags={
        "Project": "Curriculum Learning",
        "Algorithm": "CurriculumPPO",
        "Environment": tmdp.env.__class__.__name__,
        "nS": nS,
        "nA": nA,
        "gamma": gamma,
        "mu": tmdp.env.mu,
        "xi": tmdp.xi,
        "seed": seed,
        "tau": tau,
        "episodes": episodes,
        "eps_model": eps_model,
        "eps_ppo": eps_ppo,
        })
# Save the best model to MLflow

In [ ]:
study.best_params

In [ ]:
import matplotlib.pyplot as plt
reward_records = curr_PPO.reward_records
# Generate recent 50 interval average
average_reward = []
scaled_reward = []
for idx in range(len(reward_records)):
    avg_list = np.empty(shape=(1,), dtype=int)
    scaled_reward.append(reward_records[idx])
    if idx < 500:
        avg_list = reward_records[:idx+1]
    else:
        avg_list = reward_records[idx-499:idx+1]
    average_reward.append(np.mean(avg_list))
# Plot
#plt.plot(scaled_reward)
plt.plot(average_reward[:-1])